# 事前準備

In [ ]:
# Initializing
from google.colab import drive

drive.mount('/content/drive')
# Define constant value
RESEARCH_WORK_PATH = "/content/drive/My Drive/Colab Notebooks/BachelorResearch/"
DIR_NAME = 'stft_crop_0.2'


Mounted at /content/drive


## 使用ディレクトリの存在確認と作成

In [ ]:
import os
if not os.path.exists(RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/2s/" + DIR_NAME + "/"):
  os.mkdir(RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/2s/" + DIR_NAME + "/")
if not os.path.exists(RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/2s/" + DIR_NAME + "/Q1"):
  os.mkdir(RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/2s/" + DIR_NAME + "/Q1")
if not os.path.exists(RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/2s/" + DIR_NAME + "/Q2"):
  os.mkdir(RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/2s/" + DIR_NAME + "/Q2")
if not os.path.exists(RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/2s/" + DIR_NAME + "/Q3"):
  os.mkdir(RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/2s/" + DIR_NAME + "/Q3")
if not os.path.exists(RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/2s/" + DIR_NAME + "/Q4"):
  os.mkdir(RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/2s/" + DIR_NAME + "/Q4")

In [ ]:
from librosa.core.convert import midi_to_note
import os
import pathlib
import librosa
import numpy as np
from matplotlib import pyplot as plt
import cv2

def min_max(x, axis=None): # do min_max normalization ?->https://atmarkit.itmedia.co.jp/ait/articles/2110/07/news027.html
    min = 0
    max = 154.28702 
    # max = 137.74599
    x = np.abs(x)
    result = (x-min)/(max-min)
    return result

def audio_max_magnitude(filepath,n_fft = 512,hop_length = 256): # 154.28702
  try:
    y, sr = librosa.load(filepath)
    stft = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length)) # オーバラップ率　0％
    stft = to_db(stft)
    return stft.max()
  except Exception:
    return 0

def audio_min_magnitude(filepath,n_fft = 512,hop_length = 256): # -inf
  try:
    y, sr = librosa.load(filepath)
    stft = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length)) # オーバラップ率　0％
    stft = to_db(stft)
    return stft.min()
  except Exception:
    return 0

def audio_min_freq(filepath, cut_th, n_fft = 512,hop_length = 256):
  # cut_th = 0.15 # 150
  # cut_th = 1e-1 # 224
  # cut_th = 0.2  # 102
  cut_index = 0
  try:
    y, sr = librosa.load(filepath)
    stft = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length)) # オーバラップ率　0％
    stft = min_max(stft)
    for fft in stft.T:
      for i, mag in enumerate(reversed(fft)):
        if mag>cut_th:
          # cut_index = len(fft)-1-i
          if cut_index < len(fft)-1-i:
            cut_index = len(fft)-1-i
          break
    print(cut_index)
    return cut_index
  except Exception:
    return -1

def to_db(x, dBref=2e-5):
    y = 20 * np.log10(x / dBref)
    return y

def audio_stft(filepath,n_fft = 512,hop_length = 256):
  try:
    y, sr = librosa.load(filepath)
    stft = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length)) # オーバラップ率　0％
    stft = to_db(stft)
    stft = min_max(stft)
    dirs = filepath.split("/")
    output_path_str = DIR_NAME + "/" + dirs[len(dirs)-2] + '/' + dirs[len(dirs)-1] # Make end of the path ex./stft/Q1/Q1.MT0018024152_9.wav
    output_path_str = pathlib.Path(filepath).parents[1] / pathlib.Path(output_path_str) # Make head of the path ex./content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s 
    print(stft)                                                                                    # and then, add the end of the path
    print(output_path_str)
    cv2.imwrite(str(output_path_str.parent) + '/' + output_path_str.stem + '.jpg' ,stft*255) # cv2 only support uint8(0~255) format.
    # print((stft*255).shape) # -> (257, 173)
    print("Done.")
    return 
  except Exception:
    return
    
def audio_stft_crop(filepath,crop_position=102,n_fft = 512,hop_length = 256):
  try:
    y, sr = librosa.load(filepath)
    stft = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length)) # オーバラップ率　0％
    stft = min_max(stft)
    dirs = filepath.split("/")
    output_path_str = DIR_NAME + "/" + dirs[len(dirs)-2] + '/' + dirs[len(dirs)-1] # Make end of the path ex./stft/Q1/Q1.MT0018024152_9.wav
    output_path_str = pathlib.Path(filepath).parents[1] / pathlib.Path(output_path_str) # Make head of the path ex./content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s 
                                                                                        # and then, add the end of the path
    print(output_path_str)
    cv2.imwrite(str(output_path_str.parent) + '/' + output_path_str.stem + '.jpg' ,stft[:crop_position,:]*255) # cv2 only support uint8(0~255) format.
    # print((stft*255).shape) # -> (257, 173)
    print("Done.")
    return 
  except Exception:
    return
    
def path_to_audiofiles(dir_folder):
    list_of_audio = []
    for file in os.listdir(dir_folder):
        if file.endswith(".wav"):
            directory = "%s%s" % (dir_folder, file)
            list_of_audio.append(directory)
    return list_of_audio

Q1_2s = RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/2s/Q1/"
Q2_2s = RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/2s/Q2/"
Q3_2s = RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/2s/Q3/"
Q4_2s = RESEARCH_WORK_PATH + "MER_audio_taffc_dataset_wav/2s/Q4/"

# Search the maxium value of arrays
'''
Q1_max = -1
for audio in path_to_audiofiles(Q1_2s):
  tmp = audio_max_magnitude(audio)
  if (tmp > Q1_max):
    Q1_max = tmp
Q2_max = -1
for audio in path_to_audiofiles(Q2_2s):
  tmp = audio_max_magnitude(audio)
  if (tmp > Q2_max):
    Q2_max = tmp
Q3_max = -1
for audio in path_to_audiofiles(Q3_2s):
  tmp = audio_max_magnitude(audio)
  if (tmp > Q3_max):
    Q3_max = tmp
Q4_max = -1
for audio in path_to_audiofiles(Q4_2s):
  tmp = audio_max_magnitude(audio)
  if (tmp > Q4_max):
    Q4_max = tmp

max_of_all = max(Q1_max,Q2_max,Q3_max,Q4_max)
print(max_of_all)

Q1_min = 1e9
for audio in path_to_audiofiles(Q1_2s):
  tmp = audio_min_magnitude(audio)
  if (tmp < Q1_min):
    Q1_min = tmp
Q2_min = 1e9
for audio in path_to_audiofiles(Q2_2s):
  tmp = audio_min_magnitude(audio)
  if (tmp < Q2_min):
    Q2_min = tmp
Q3_min = 1e9
for audio in path_to_audiofiles(Q3_2s):
  tmp = audio_min_magnitude(audio)
  if (tmp < Q3_min):
    Q3_min = tmp
Q4_min = 1e9
for audio in path_to_audiofiles(Q4_2s):
  tmp = audio_min_magnitude(audio)
  if (tmp < Q4_min):
    Q4_min = tmp

min_of_all = min(Q1_min,Q2_min,Q3_min,Q4_min)
print(min_of_all)
'''



'''
max_ = -1
cut_th = 0.2

for audio in path_to_audiofiles(Q1_2s):
  tmp = audio_min_freq(audio, cut_th)
  if tmp > max_:
    max_ = tmp
for audio in path_to_audiofiles(Q2_2s):
  tmp = audio_min_freq(audio, cut_th)
  if tmp > max_:
    max_ = tmp
for audio in path_to_audiofiles(Q3_2s):
  tmp = audio_min_freq(audio, cut_th)
  if tmp > max_:
    max_ = tmp
for audio in path_to_audiofiles(Q4_2s):
  tmp = audio_min_freq(audio, cut_th)
  if tmp > max_:
    max_ = tmp
print(max_)
'''

'\nmax_ = -1\ncut_th = 0.2\n\nfor audio in path_to_audiofiles(Q1_2s):\n  tmp = audio_min_freq(audio, cut_th)\n  if tmp > max_:\n    max_ = tmp\nfor audio in path_to_audiofiles(Q2_2s):\n  tmp = audio_min_freq(audio, cut_th)\n  if tmp > max_:\n    max_ = tmp\nfor audio in path_to_audiofiles(Q3_2s):\n  tmp = audio_min_freq(audio, cut_th)\n  if tmp > max_:\n    max_ = tmp\nfor audio in path_to_audiofiles(Q4_2s):\n  tmp = audio_min_freq(audio, cut_th)\n  if tmp > max_:\n    max_ = tmp\nprint(max_)\n'

In [ ]:
for audio in path_to_audiofiles(Q1_2s):
  audio_stft_crop(audio)

ストリーミング出力は最後の 5000 行に切り捨てられました。
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q1/Q1.MT0012798988_2.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q1/Q1.MT0012798988_1.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q1/Q1.MT0012798988_3.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q1/Q1.MT0012798988_4.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q1/Q1.MT0012798988_5.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q1/Q1.MT0012798988_6.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q1/Q1.MT0012798988_7.wav
Done.
/content/drive/My Drive/Colab Noteboo

In [ ]:
for audio in path_to_audiofiles(Q2_2s):
  audio_stft_crop(audio)

ストリーミング出力は最後の 5000 行に切り捨てられました。
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q2/Q2.MT0008733057_1.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q2/Q2.MT0008733057_3.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q2/Q2.MT0008733057_2.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q2/Q2.MT0008733057_5.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q2/Q2.MT0008733057_4.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q2/Q2.MT0008733057_6.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q2/Q2.MT0008733057_8.wav
Done.
/content/drive/My Drive/Colab Noteboo

In [ ]:
for audio in path_to_audiofiles(Q3_2s):
  audio_stft_crop(audio)

ストリーミング出力は最後の 5000 行に切り捨てられました。
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q3/Q3.MT0030304982_1.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q3/Q3.MT0030304982_2.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q3/Q3.MT0030304982_3.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q3/Q3.MT0030304982_5.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q3/Q3.MT0030304982_4.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q3/Q3.MT0030304982_6.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q3/Q3.MT0030304982_8.wav
Done.
/content/drive/My Drive/Colab Noteboo

In [ ]:
for audio in path_to_audiofiles(Q4_2s):
  audio_stft_crop(audio)

ストリーミング出力は最後の 5000 行に切り捨てられました。
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q4/Q4.MT0001195535_14.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q4/Q4.MT0005608787_2.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q4/Q4.MT0005608787_4.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q4/Q4.MT0005608787_5.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q4/Q4.MT0005608787_3.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q4/Q4.MT0005608787_8.wav
Done.
/content/drive/My Drive/Colab Notebooks/BachelorResearch/MER_audio_taffc_dataset_wav/2s/stft_crop_0.2/Q4/Q4.MT0005608787_6.wav
Done.
/content/drive/My Drive/Colab Notebo

In [ ]:
def count_file(folder_path):

  import pathlib
  initial_count = 0
  for path in pathlib.Path(folder_path).iterdir():
    if path.is_file():
      initial_count += 1

  return(initial_count)

In [ ]:
# How many datas?    
print("[INFO] Datas in /2s/" + DIR_NAME + "/Q1: ", end='')
print(count_file(RESEARCH_WORK_PATH + "/MER_audio_taffc_dataset_wav/2s/" + DIR_NAME + "/Q1"))
print("[INFO] Datas in /2s/" + DIR_NAME + "/Q2: ", end='')
print(count_file(RESEARCH_WORK_PATH + "/MER_audio_taffc_dataset_wav/2s/" + DIR_NAME + "/Q2"))
print("[INFO] Datas in /2s/" + DIR_NAME + "/Q3: ", end='')
print(count_file(RESEARCH_WORK_PATH + "/MER_audio_taffc_dataset_wav/2s/" + DIR_NAME + "/Q3"))
print("[INFO] Datas in /2s/" + DIR_NAME + "/Q4: ", end='')
print(count_file(RESEARCH_WORK_PATH + "/MER_audio_taffc_dataset_wav/2s/" + DIR_NAME + "/Q4"))

[INFO] Datas in /2s/stft_crop_0.2/Q1: 2800
[INFO] Datas in /2s/stft_crop_0.2/Q2: 2800
[INFO] Datas in /2s/stft_crop_0.2/Q3: 2800
[INFO] Datas in /2s/stft_crop_0.2/Q4: 2800
